### ***IMPORTING NECESSARY LIBRARIES***

In [9]:
import time
from Crypto.PublicKey import RSA
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.Random import get_random_bytes
from tinyec import registry
import hashlib
import secrets

### ***ECC KEY PAIR GENEARTION***

The function ***generate_ecc_key_pair*** creates a private-public key pair for elliptic curve cryptography. It begins by selecting the ***brainpoolP256r1*** curve, known for its robust security features. The private key is a randomly chosen number within the curve's field size, ensuring unpredictability and cryptographic strength. The public key is calculated by multiplying the private key with the curve's ***generator point (g)***, producing a point on the curve represented by its (x, y) coordinates. These keys are essential for encryption, decryption, and shared key generation. The function prints both keys to demonstrate the process and returns them for further use.

In [10]:
def generate_ecc_key_pair():
    print("\n[STEP 1: ECC Key Pair Generation]")
    curve = registry.get_curve('brainpoolP256r1')
    print(f"ECC Curve: {curve.name}")
    private_key = secrets.randbelow(curve.field.n)
    public_key = private_key * curve.g
    print(f"ECC Private Key: {private_key}")
    print(f"ECC Public Key: ({public_key.x}, {public_key.y})")
    return private_key, public_key

### ***RSA KEY PAIR GENERATION***

The ***generate_rsa_key_pair*** function generates RSA public and private keys. It uses the ***Crypto.PublicKey.RSA.generate*** method to create a key pair with a specified bit size (commonly 2048 or 4096 bits, depending on the desired security level). The generated keys include both the modulus (used in encryption and decryption) and the exponents (public and private). The function also provides functionality to export these keys in PEM format, a widely used standard for storing and sharing cryptographic keys. This allows easy integration with other systems that require RSA for secure communication or signing.

In [11]:
def generate_rsa_key_pair():
    print("\n[STEP 2: RSA Key Pair Generation]")
    key = RSA.generate(2048)
    private_key = key.export_key()
    public_key = key.publickey().export_key()
    print(f"Generated RSA Modulus (n): {key.n}")
    print(f"Generated RSA Public Exponent (e): {key.e}")
    print(f"Generated RSA Private Exponent (d): {key.d}")
    print(f"Truncated RSA Private Key: {private_key.decode()[:50]}...")
    print(f"Truncated RSA Public Key: {public_key.decode()[:50]}...")
    return private_key, public_key

### ***ECC SHARED KEY DERIVATION***

The ***ecc_point_to_256_bit_key*** function derives a shared 256-bit key from a point on an elliptic curve. This is a critical step in ***elliptic curve Diffie-Hellman (ECDH)*** key exchange, where two parties independently compute the same shared secret using their private key and the other party's public key. The function hashes the x coordinate of the elliptic curve point using ***SHA-256*** to produce a 256-bit key. This ensures the shared secret is both compact and suitable for use in symmetric encryption, such as AES.

In [12]:
def ecc_point_to_256_bit_key(point):
    print("\n[STEP 3: ECC Shared Key Derivation]")
    print(f"ECC Shared Point Coordinates: ({point.x}, {point.y})")
    sha = hashlib.sha256(int.to_bytes(point.x, 32, 'big'))
    sha.update(int.to_bytes(point.y, 32, 'big'))
    derived_key = sha.digest()
    print(f"Derived ECC 256-bit Key: {derived_key.hex()}")
    return derived_key

### ***MESSAGE ENCRYPTION***

The ***encrypt_message*** function demonstrates hybrid encryption by combining ECC, RSA, and AES. First, a symmetric AES key is generated for encrypting the actual message, as AES is efficient for encrypting large data. This AES key is then securely encrypted using both the ECC public key (for ECDH key exchange) and the RSA public key. The encrypted AES key, along with the AES-encrypted message, is returned as part of the final ciphertext. This approach ensures that the message remains secure while leveraging the strengths of ECC and RSA for key management.

In [13]:
def encrypt_message(msg, ecc_public_key, rsa_public_key):
    print("\n[STEP 4: Message Encryption]")

    # ECC Key Exchange
    print("\n4.1: Performing ECC Key Exchange...")
    curve = registry.get_curve('brainpoolP256r1')
    ecc_private_key = secrets.randbelow(curve.field.n)
    print(f"Ephemeral ECC Private Key: {ecc_private_key}")
    shared_ecc_key = ecc_private_key * ecc_public_key
    secret_key = ecc_point_to_256_bit_key(shared_ecc_key)

    # AES Encryption
    print("\n4.2: Encrypting Message with AES...")
    aes_key = secret_key  # Use ECC-derived key as AES key
    print(f"Using ECC-Derived AES Key: {aes_key.hex()}")
    cipher_aes = AES.new(aes_key, AES.MODE_GCM)
    ciphertext, tag = cipher_aes.encrypt_and_digest(msg.encode('utf-8'))
    print(f"AES Nonce: {cipher_aes.nonce.hex()}")
    print(f"AES Ciphertext: {ciphertext.hex()}")
    print(f"AES Tag: {tag.hex()}")

    # RSA Encryption of AES Key
    print("\n4.3: Encrypting AES Key with RSA...")
    cipher_rsa = PKCS1_OAEP.new(RSA.import_key(rsa_public_key))
    enc_aes_key = cipher_rsa.encrypt(aes_key)
    print(f"RSA Encrypted AES Key: {enc_aes_key.hex()}")

    return (cipher_aes.nonce, tag, ciphertext, enc_aes_key)

### ***MESSAGE DECRYPTION***

The ***decrypt_message*** function implements the reverse of the encryption process. It begins by decrypting the AES key using both the ECC private key (to compute the shared key) and the RSA private key. Once the AES key is retrieved, it is used to decrypt the message ciphertext back into its original plaintext. This hybrid decryption process ensures that the encrypted message remains secure, as it can only be decrypted by someone possessing the correct ECC and RSA private keys.

In [14]:
def decrypt_message(enc_msg, ecc_private_key, rsa_private_key):
    print("\n[STEP 5: Message Decryption]")

    # ECC Key Exchange
    print("\n5.1: Performing ECC Key Exchange...")
    nonce, tag, ciphertext, enc_aes_key = enc_msg

    # RSA Decryption of AES Key
    print("\n5.2: Decrypting AES Key with RSA...")
    cipher_rsa = PKCS1_OAEP.new(RSA.import_key(rsa_private_key))
    aes_key = cipher_rsa.decrypt(enc_aes_key)
    print(f"Decrypted AES Key (from RSA): {aes_key.hex()}")

    # AES Decryption
    print("\n5.3: Decrypting Message with AES...")
    cipher_aes = AES.new(aes_key, AES.MODE_GCM, nonce=nonce)
    plaintext = cipher_aes.decrypt_and_verify(ciphertext, tag)
    print(f"Decrypted Plaintext: {plaintext.decode('utf-8')}")

    return plaintext.decode('utf-8')

### ***EXAMPLE***

In [15]:
# Example
print("[STEP 0: Key Generation Phase]")
time.sleep(1)
ecc_private_key, ecc_public_key = generate_ecc_key_pair()
rsa_private_key, rsa_public_key = generate_rsa_key_pair()

message = input("\nEnter a message to encrypt: ")

# Encryption process
encrypted_msg = encrypt_message(message, ecc_public_key, rsa_public_key)

# Decryption process
decrypted_msg = decrypt_message(encrypted_msg, ecc_private_key, rsa_private_key)

print("\n[FINAL RESULTS]")
print(f"Original Message: {message}")
print(f"Decrypted Message: {decrypted_msg}")

[STEP 0: Key Generation Phase]

[STEP 1: ECC Key Pair Generation]
ECC Curve: brainpoolP256r1
ECC Private Key: 56604172425792413453937508513151751284074395760972680127861052531589191858107
ECC Public Key: (48402493101293374281022664284399145291394526154629406288764133608612003299469, 49570557666221312746827297078903035009393236379962279109146255841911126496741)

[STEP 2: RSA Key Pair Generation]
Generated RSA Modulus (n): 19086817894235053984063013653170137317313842743976747367406692817198739406378212030096377179830095308879003873305098082680448559989467754542430481453786553891179708602774338328042342482972053956125201694031629572019074183606242997673581732512330841352666826191344498024387728360935678618465662074022605075159132848866125434603503760823702917005273186616196434255653935686897367308770348218003117602775483300748764782498182376932255310052600677114149492638280578712264785678136954056822956419253562194557728526961679882592334701476828774276952650573883947593650649646309882958